# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [17]:
def priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status):
    """
    Calculates the PRIEST COVID-19 Clinical Severity Score based on the provided parameters.

    Args:
    sex (str): Gender assigned at birth.
    age (int): Age in years.
    respiratory_rate (int): Respiratory rate in breaths per minute.
    oxygen_saturation (float): Oxygen saturation as a percent between 0 and 1.
    heart_rate (int): Heart rate in beats per minute.
    systolic_bp (int): Systolic BP in mmHg.
    temperature (float): Temperature in degrees Celsius.
    alertness (str): Alertness as a string description.
    inspired_oxygen (str): Inspired Oxygen as a string description.
    performance_status (str): Performance Status as a string description.

    Returns:
    float or None: The computed risk percentage as a numeric value between 0 and 1. Returns None if any input is invalid.

    >>> priest('male', 40, 24, 0.96, 105, 115, 34.9, 'alert', 'air', 'unrestricted normal acitivity')
    0.15
    >>> priest('male',51, 22, 0.95, 42.0, 94, 35.1, 'confused or not alert', 'supplemental oxygen', 'limited self-care')
    0.59
    >>> priest('male', 95, 30, 0.91, 150 , 300, 42, 'confused or not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
    0.99
    """

    # Convert input strings to lowercase for case insensitivity
    sex = sex.lower()
    alertness = alertness.lower()
    inspired_oxygen = inspired_oxygen.lower()
    performance_status = performance_status.lower()

    # Scoring based on the provided algorithm
    score = 0

    # Scoring for sex
    if sex == 'male':
        score += 1
    elif sex == 'female':
        score += 0
    else:
        return None  # If sex is neither male nor female, return None

    # Scoring for age
    if 16 <= age < 50:
        score += 0
    elif 50 <= age <= 65:
        score += 2
    elif 66 <= age <= 80:
        score += 3
    elif age > 80:
        score += 4
    else:
        return None

    # Scoring for respiratory rate
    if respiratory_rate < 9:
        score += 3
    elif 9 <= respiratory_rate <= 11:
        score += 1
    elif 21 <= respiratory_rate <= 24:
        score += 3
    elif respiratory_rate > 24:
        score += 3
    else:
        return None

    # Scoring for oxygen saturation
    if oxygen_saturation > 95:
        score += 0
    elif 94 <= oxygen_saturation <= 95:
        score += 1
    elif 92 <= oxygen_saturation <= 93:
        score += 2
    elif oxygen_saturation < 92:
        score += 3
    else:
        return None

    # Scoring for heart rate
    if heart_rate < 41:
        score += 3
    elif 41 <= heart_rate <= 50:
        score += 1
    elif 91 <= heart_rate <= 110:
        score += 1
    elif 111 <= heart_rate <= 130:
        score += 2
    elif heart_rate > 130:
        score += 3
    else:
        return None

    # Scoring for systolic blood pressure
    if systolic_bp < 91:
        score += 3
    elif 91 <= systolic_bp <= 100:
        score += 2
    elif 101 <= systolic_bp <= 110:
        score += 1
    elif systolic_bp > 219:
        score += 3
    else:
        return None

    # Scoring for temperature
    if temperature < 35.1:
        score += 3
    elif 35.1 <= temperature <= 36.0:
        score += 1
    elif 38.1 <= temperature <= 39.0:
        score += 1
    elif temperature > 39.0:
        score += 2
    else:
        return None

    # Scoring for alertness
    if alertness == 'alert':
        score += 0
    elif alertness == 'confused or not alert':
        score += 3
    else:
        return None  # If alertness is not recognized, return None

    # Scoring for inspired oxygen
    if inspired_oxygen == 'air':
        score += 0
    elif inspired_oxygen == 'supplemental oxygen':
        score += 2
    else:
        return None  # If inspired oxygen is not recognized, return None

    # Scoring for performance status
    if performance_status == 'unrestricted normal activity':
        score += 0
    elif performance_status == 'limited strenuous activity, can do light activity':
        score += 1
    elif performance_status == 'limited activity, can self-care':
        score += 2
    elif performance_status == 'limited self-care':
        score += 3
    elif performance_status == 'bed/chair bound, no self-care':
        score += 4
    else:
        return None  # If performance status is not recognized, return None

    # Calculate the total PRIEST score
    priest_score = score

    # Converting the PRIEST score to 30-day probability of an outcome
    if priest_score == 0:
        risk_percentage = 0.01
    elif priest_score == 1:
        risk_percentage = 0.01
    elif priest_score == 2:
        risk_percentage = 0.02
    elif priest_score == 3:
        risk_percentage = 0.02
    elif priest_score == 4:
        risk_percentage = 0.03
    elif priest_score == 5:
        risk_percentage = 0.09
    elif priest_score == 6:
        risk_percentage = 0.15
    elif priest_score == 7:
        risk_percentage = 0.18
    elif priest_score == 8:
        risk_percentage = 0.22
    elif priest_score == 9:
        risk_percentage = 0.26
    elif priest_score == 10:
        risk_percentage = 0.29
    elif priest_score == 11:
        risk_percentage = 0.34
    elif priest_score == 12:
        risk_percentage = 0.38
    elif priest_score == 13:
        risk_percentage = 0.46
    elif priest_score == 14:
        risk_percentage = 0.47
    elif priest_score == 15:
        risk_percentage = 0.49
    elif priest_score == 16:
        risk_percentage = 0.55
    elif 17 <= priest_score <= 25:
        risk_percentage = 0.59
    elif priest_score >= 26:
        risk_percentage = 0.99
    else:
        return None

    return risk_percentage



In [18]:
import doctest
doctest.run_docstring_examples(priest, globals(), verbose=True)

Finding tests in NoName
Trying:
    priest('male', 40, 24, 0.96, 105, 115, 34.9, 'alert', 'air', 'unrestricted normal acitivity')
Expecting:
    0.15
**********************************************************************
File "__main__", line 20, in NoName
Failed example:
    priest('male', 40, 24, 0.96, 105, 115, 34.9, 'alert', 'air', 'unrestricted normal acitivity')
Expected:
    0.15
Got nothing
Trying:
    priest('male',51, 22, 0.95, 42.0, 94, 35.1, 'confused or not alert', 'supplemental oxygen', 'limited self-care')
Expecting:
    0.59
ok
Trying:
    priest('male', 95, 30, 0.91, 150 , 300, 42, 'confused or not alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
Expecting:
    0.99
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [3]:
import requests

In [4]:
hospitals = requests.get('https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1')

In [5]:
print (hospitals.text)

{"age": "40", "sex": "male", "risk": "0.1", "hospital": "Southwest Hospital and Medical Center"}


In [6]:
def find_hospital(age, sex, risk):
    """
    Calls a REST web service to find a hospital based on the provided age, sex, and risk percentage.

    Args:
    age (int): Age of the patient.
    sex (str): Gender of the patient, either 'male' or 'female'.
    risk (float): The risk percentage of the patient.

    Returns:
    str or None: The name of the hospital where the patient should be sent. Returns None if anything fails.

    >>> find_hospital(90, 'male', 0.1)
    'Wesley Woods Geriatric Hospital'

    >>> find_hospital(60, 'female', 0.5)
    'Emory Dunwoody Medical Center'

    >>> find_hospital(20, 'male', 0.8)
    'Emory Dunwoody Medical Center'

    >>> find_hospital(80, 'Female', 0.3)
    'Wesley Woods Geriatric Hospital'
    """

    hospitals = 'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?'
    json = f'age={age}&sex={sex}&risk_pct={risk}'

    response = requests.get(hospitals + json)
    if response.status_code == 200:
        data = response.json()
        return data.get('hospital')
    else:
        return None


In [7]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(90, 'male', 0.1)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital(60, 'female', 0.5)
Expecting:
    'Emory Dunwoody Medical Center'
ok
Trying:
    find_hospital(20, 'male', 0.8)
Expecting:
    'Emory Dunwoody Medical Center'
ok
Trying:
    find_hospital(80, 'Female', 0.3)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [8]:
import requests

In [9]:
address_hospital = requests.get('https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-')

In [10]:
print (address_hospital)

<Response [200]>


In [11]:
hospitals = requests.get('https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1')

In [12]:
def get_address(hospital_name):
    """
    Retrieves the address for the given hospital name from a JSON file.

    Args:
    hospital_name (str): Name of the hospital.

    Returns:
    str or None: The address of the hospital. Returns None if the hospital name is not found.
    >>> get_address('Southwest Hospital and Medical Center')

    >>> get_address('East General Hospital')
    """

    address_hospital = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"
    response = requests.get(address_hospital)
    if response.status_code == 200:
        data = response.json()
        for hospital in data['address_hospital']:
            if hospital['hospitals'] == hospital_name:
                return hospital['ADDRESS']
        return None  # Hospital name not found in the JSON data
    else:
        return None  # Failed to retrieve JSON data

# Testing the get_address function with some sample test cases
print(get_address('Southwest Hospital and Medical Center'))  # '123 Main Street, Atlanta, GA'
print(get_address('East General Hospital'))  # '456 Elm Street, Atlanta, GA'
print(get_address("North Children's Hospital"))  # '789 Oak Street, Atlanta, GA'
print(get_address('West Senior Hospital'))  # '321 Pine Street, Atlanta, GA'

KeyError: ignored

In [ ]:
import doctest
doctest.run_docstring_examples(address_hospital, globals(),verbose=True)

## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [ ]:
import requests

In [ ]:
process_people = requests.get('https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh')

In [ ]:
from io import StringIO

def risk_calculator(sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired):
    # Your risk calculator function code here
    pass

def process_people(file_location):
    """
    Process the people.psv file and calculate the risk for each individual.

    Args:
    file_location (str): The URL of the .psv file containing patient information.

    Returns:
    list: A list of dictionaries where each dictionary represents a patient's information along with calculated risk.
    """
    response = requests.get(file_location)
    data = StringIO(response.text)

    result = []
    for line in data:
        if line.strip():  # Skips empty lines
            row = line.strip().split("|")
            patient_number = row[0]
            sex = row[1].upper()
            breath = int(row[3])
            o2sat = float(row[4])
            heart = int(row[5])
            systolic = int(row[6])
            temp = float(row[7])
            alertness = row[8].upper()
            inspired = row[9].upper()

            # Call risk_calculator function
            status, hospital, address = risk_calculator(sex, breath, o2sat, heart, systolic, temp, alertness, inspired)

            result.append({
                patient_number: [sex, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]
            })

    return result

# Example usage:
# file_location = 'https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh'
# result = process_people(file_location)
# for item in result:
#     print(item)


## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [ ]:
import json
import requests

# Your process_people function here


# Fetching the people_results.json file
results_url = "https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"
response = requests.get(results_url)
data = response.json()

# Running process_people function to get calculated results
file_location = 'https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh'
calculated_results = process_people(file_location)

# Comparing the results
for i in range(len(data)):
    if data[i] != calculated_results[i]:
        print(f"Mismatch found at index {i}:")
        print("Expected:", data[i])
        print("Calculated:", calculated_results[i])
        print("\n")
print("Comparison completed.")


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---